Import necessary packages:

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from dstapi import DstApi # Denmark Statistics API wrapper

# Import our function
from data_proj import prediction


C:\Users\echic\AppData\Local\Temp\ipykernel_14640\2017622859.py:4: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


Check paths existence on computer:

In [3]:
import os 

# Using assert to check that paths exist on computer.
assert os.path.isfile('data/FU07_cp.xlsx')

# Print everything in data
os.listdir('data/')

['FU07_cp.xlsx']

Create objects to interact with API of Denmark Statistics and display a summary table of the datasets informations for the GDP dataset:

In [4]:
gdp_dst = DstApi('NRHP')

t_gdp = gdp_dst.tablesummary(language='en')
display(t_gdp)

Table NRHP: 1-2.1.1 Production, GDP and generation of income by region, transaction, price unit and time
Last update: 2022-10-27T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,OMRÅDE,18,000,All Denmark,999,Outside regions,False
1,TRANSAKT,9,P1K,P.1 Output,B2A3GD,B.2g+B.3g Gross operating surplus and mixed in...,False
2,PRISENHED,4,V_T,"Current prices, (mill. DKK.)",LRG_C,"Pr. capita, 2010-prices, chained values, (1000...",False
3,Tid,29,1993,1993,2021,2021,True


Look up values (rows) that each variable (columns) can take: 

In [5]:
for variable in t_gdp['variable name']:
    print(variable+':')
    display(gdp_dst.variable_levels(variable, language='en'))

OMRÅDE:


,id,text
0,000,All Denmark
1,084,Region Hovedstaden
2,01,Province Byen København
3,02,Province Københavns omegn
4,03,Province Nordsjælland
5,04,Province Bornholm
6,085,Region Sjælland
7,05,Province Østsjælland
8,06,Province Vest- og Sydsjælland
9,083,Region Syddanmark


TRANSAKT:


,id,text
0,P1K,P.1 Output
1,P2D,P.2 Intermediate consumption
2,B1GD,B.1g Gross value added
3,D21X31D,D.21-D.31 Taxes less subsidies on products
4,B1GQD,B.1*g Gross domestic product
5,D29X39D,D.29-D.39 Other taxes less subsidies on produc...
6,B1GFD,B.1GF Gross domestic product at factor cost
7,D1D,D.1 Compensation of employees
8,B2A3GD,B.2g+B.3g Gross operating surplus and mixed in...


PRISENHED:


,id,text
0,V_T,"Current prices, (mill. DKK.)"
1,V_C,"Pr. capita. Current prices, (1000 DKK.)"
2,LRG_T,"2010-prices, chained values, (mill. DKK.)"
3,LRG_C,"Pr. capita, 2010-prices, chained values, (1000..."


Tid:


,id,text
0,1993,1993
1,1994,1994
2,1995,1995
3,1996,1996
4,1997,1997
5,1998,1998
6,1999,1999
7,2000,2000
8,2001,2001
9,2002,2002


Look up the format of the dictionary of dataset parameters:

In [6]:
par_gdp = gdp_dst._define_base_params(language='en')

display(par_gdp)

{'table': 'nrhp',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['*']},
  {'code': 'TRANSAKT', 'values': ['*']},
  {'code': 'PRISENHED', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

Define parameters dictionary to select only specified values (rows) of dataset:

In [7]:
par_gdp = {'table': 'nrhp',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['000']},
  {'code': 'TRANSAKT', 'values': ['B1GQD']},
  {'code': 'PRISENHED', 'values': ['V_C']},
  {'code': 'Tid', 'values': ['>1993<=2021']}]}

# Just took some random parameters: I'll fix it later
# !! Does real price in gdp dataset have same base year as in consumption dataset?

Download dataset using only specified parameters:

In [8]:
gdp = gdp_dst.get_data(params=par_gdp)

display(gdp.head(5))

,OMRÅDE,TRANSAKT,PRISENHED,TID,INDHOLD
0,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1994,191
1,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1995,198
2,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1996,207
3,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1997,217
4,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1998,224


Renaming columns:

In [9]:
gdp.rename(columns = {'OMRÅDE':'Area', 
                      'PRISENHED':'Price unit', 
                      'TID':'variables', #helpfull later
                      'INDHOLD':'GDP'}, inplace=True)
gdp.head(5)

,Area,TRANSAKT,Price unit,variables,GDP
0,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1994,191
1,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1995,198
2,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1996,207
3,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1997,217
4,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1998,224


Dropping unimportant variables:

In [10]:
gdp.drop(['TRANSAKT', 'Area', 'Price unit'], axis='columns', inplace=True)
gdp.head(5)

,variables,GDP
0,1994,191
1,1995,198
2,1996,207
3,1997,217
4,1998,224


Make column names a mix of text and numbers (without spaces) and set index:

In [11]:
import string 
for value in gdp['variables'].values:
    gdp.loc[gdp['variables'].values == value,['variables']] = 'value'+str(value)
gdp = gdp.set_index('variables')
gdp.head(5)

,GDP
variables,
value1994,191
value1995,198
value1996,207
value1997,217
value1998,224


Transpose:

In [12]:
gdp = gdp.T
gdp.head(5)

variables,value1994,value1995,value1996,value1997,value1998,value1999,value2000,value2001,value2002,value2003,...,value2012,value2013,value2014,value2015,value2016,value2017,value2018,value2019,value2020,value2021
GDP,191,198,207,217,224,233,249,256,262,267,...,339,344,351,358,368,380,389,397,399,428


Import dataset for "consumption choices" previously downloaded from DST, selecting only the necessary parameters. We also skip empty rows:

In [13]:
filename = 'data/FU07_cp.xlsx'
cop = pd.read_excel(filename, skiprows=2, skipfooter=2)
display(cop)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,1994,1995,1996,1997,1998,1999,2000,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Current prices,Average Households,01.1 Food,22325,23280,23488,24021,24904,24905,25605,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
1,NaN,NaN,"04.5 Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
2,NaN,NaN,05.2 Household textiles,920,916,852,870,851,907,915,...,1278,1215,1275,1496,1307,994,1064,1116,1408,1723
3,NaN,NaN,07.1 Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
4,NaN,NaN,07.3 Transport services,3663,3701,3445,3720,3425,3669,3869,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
5,NaN,NaN,12.1 Personal care,4092,4274,4531,4697,4874,5072,5169,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
6,NaN,NaN,12.5 Insurance,5875,6206,6528,7519,8344,8809,8399,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317


We drop NaN columns:

In [14]:
drop_these = ['Unnamed: ' + str(num) for num in range(2)] # use list comprehension to create list of columns
cop.drop(drop_these, axis=1, inplace=True) # axis = 1 -> columns, inplace=True -> changed, no copy made
cop.head(10)

,Unnamed: 2,1994,1995,1996,1997,1998,1999,2000,2001,2002,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,01.1 Food,22325,23280,23488,24021,24904,24905,25605,25803,25972,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
1,"04.5 Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
2,05.2 Household textiles,920,916,852,870,851,907,915,1071,1272,...,1278,1215,1275,1496,1307,994,1064,1116,1408,1723
3,07.1 Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
4,07.3 Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
5,12.1 Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
6,12.5 Insurance,5875,6206,6528,7519,8344,8809,8399,8292,8207,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317


Renaming consumption and year column names:

In [15]:
cop.rename(columns = {'Unnamed: 2':'variables'}, inplace=True)

col_dict = {}
col_dict = {str(i) : f'value{i}' for i in range(1994,2021+1)}
cop.rename(columns = col_dict, inplace=True)

cop.head(5)

,variables,value1994,value1995,value1996,value1997,value1998,value1999,value2000,value2001,value2002,...,value2012,value2013,value2014,value2015,value2016,value2017,value2018,value2019,value2020,value2021
0,01.1 Food,22325,23280,23488,24021,24904,24905,25605,25803,25972,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
1,"04.5 Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
2,05.2 Household textiles,920,916,852,870,851,907,915,1071,1272,...,1278,1215,1275,1496,1307,994,1064,1116,1408,1723
3,07.1 Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
4,07.3 Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026


Dropping unimportant variables:

In [16]:
# Build up a logical index I
I = cop.variables.str.contains('Household textiles')
cop.loc[I, :]
cop = cop.loc[I == False] # keeping everything else
cop.head(10)

,variables,value1994,value1995,value1996,value1997,value1998,value1999,value2000,value2001,value2002,...,value2012,value2013,value2014,value2015,value2016,value2017,value2018,value2019,value2020,value2021
0,01.1 Food,22325,23280,23488,24021,24904,24905,25605,25803,25972,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
1,"04.5 Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
3,07.1 Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
4,07.3 Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
5,12.1 Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
6,12.5 Insurance,5875,6206,6528,7519,8344,8809,8399,8292,8207,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317


Resetting index:

In [17]:
cop.reset_index(inplace = True, drop = True) # Drop old index too
cop.iloc[0:7,:]

,variables,value1994,value1995,value1996,value1997,value1998,value1999,value2000,value2001,value2002,...,value2012,value2013,value2014,value2015,value2016,value2017,value2018,value2019,value2020,value2021
0,01.1 Food,22325,23280,23488,24021,24904,24905,25605,25803,25972,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
1,"04.5 Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
2,07.1 Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
3,07.3 Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
4,12.1 Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
5,12.5 Insurance,5875,6206,6528,7519,8344,8809,8399,8292,8207,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317


Removing numbers from consumption categories:

In [18]:
import string 
for value in cop['variables'].values:
    cop.loc[cop['variables'].values == value,['variables']] = value.strip('0123456789.')

cop.loc[0,['variables']] = 'Total consumption'

cop

,variables,value1994,value1995,value1996,value1997,value1998,value1999,value2000,value2001,value2002,...,value2012,value2013,value2014,value2015,value2016,value2017,value2018,value2019,value2020,value2021
0,Total consumption,22325,23280,23488,24021,24904,24905,25605,25803,25972,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
1,"Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
2,Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
3,Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
4,Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
5,Insurance,5875,6206,6528,7519,8344,8809,8399,8292,8207,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317


Set Variables as Index:

In [19]:
cop = cop.set_index('variables')
cop

,value1994,value1995,value1996,value1997,value1998,value1999,value2000,value2001,value2002,value2003,...,value2012,value2013,value2014,value2015,value2016,value2017,value2018,value2019,value2020,value2021
variables,,,,,,,,,,,,,,,,,,,,,
Total consumption,22325,23280,23488,24021,24904,24905,25605,25803,25972,26428,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
"Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,17939,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,10423,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,4229,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,5314,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
Insurance,5875,6206,6528,7519,8344,8809,8399,8292,8207,10671,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317


Concatenate gdp and cop datasets:


In [20]:
#Check if they have the same variables 
different_years = [y for y in cop.columns.unique() if y not in gdp.columns.unique()] 
print(f'Columns (years) found in cop data but not in gdp: {different_years}')

#Concatenate them
all = pd.concat([cop,gdp])
all

Columns (years) found in cop data but not in gdp: []


,value1994,value1995,value1996,value1997,value1998,value1999,value2000,value2001,value2002,value2003,...,value2012,value2013,value2014,value2015,value2016,value2017,value2018,value2019,value2020,value2021
Total consumption,22325,23280,23488,24021,24904,24905,25605,25803,25972,26428,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
"Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,17939,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,10423,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,4229,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,5314,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
Insurance,5875,6206,6528,7519,8344,8809,8399,8292,8207,10671,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317
GDP,191,198,207,217,224,233,249,256,262,267,...,339,344,351,358,368,380,389,397,399,428


Rename index:

In [21]:
all.index.names = ['variables']
all

,value1994,value1995,value1996,value1997,value1998,value1999,value2000,value2001,value2002,value2003,...,value2012,value2013,value2014,value2015,value2016,value2017,value2018,value2019,value2020,value2021
variables,,,,,,,,,,,,,,,,,,,,,
Total consumption,22325,23280,23488,24021,24904,24905,25605,25803,25972,26428,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
"Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,17939,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,10423,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,4229,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,5314,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
Insurance,5875,6206,6528,7519,8344,8809,8399,8292,8207,10671,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317
GDP,191,198,207,217,224,233,249,256,262,267,...,339,344,351,358,368,380,389,397,399,428


Homogeneity in units:
Consumption is in DKK while GDP (per capita) is in 1000 DKK. It will be homogenized towards the unitary value.

In [22]:
scalar = 1000
# all[all.index == "GDP"] = all[all.index == "GDP"] * scalar
# all

# Alternative: making consumption in 1000 DKK units
for i in all.index.values:
    if i != 'GDP':
        all[all.index == i] = all[all.index == i] / scalar

all

,value1994,value1995,value1996,value1997,value1998,value1999,value2000,value2001,value2002,value2003,...,value2012,value2013,value2014,value2015,value2016,value2017,value2018,value2019,value2020,value2021
variables,,,,,,,,,,,,,,,,,,,,,
Total consumption,22.325,23.280,23.488,24.021,24.904,24.905,25.605,25.803,25.972,26.428,...,33.133,32.802,33.381,33.393,33.268,33.526,35.068,34.380,35.485,34.864
"Electricity, gas and other fuels",11.551,11.786,13.261,14.056,14.669,14.579,15.426,17.058,17.774,17.939,...,23.866,24.282,24.322,22.306,20.812,23.919,23.940,23.580,19.925,25.635
Purchase of vehicles,12.170,12.029,12.453,13.525,13.535,14.268,10.072,9.742,10.985,10.423,...,14.669,15.307,13.928,17.255,15.671,15.465,17.742,17.754,21.668,20.965
Transport services,3.663,3.701,3.445,3.720,3.425,3.669,3.869,3.984,4.154,4.229,...,6.188,6.096,5.820,4.841,4.533,5.133,6.010,5.918,3.324,3.026
Personal care,4.092,4.274,4.531,4.697,4.874,5.072,5.169,5.017,5.210,5.314,...,6.163,6.236,5.916,6.283,5.945,6.522,6.733,6.155,5.970,5.761
Insurance,5.875,6.206,6.528,7.519,8.344,8.809,8.399,8.292,8.207,10.671,...,15.398,14.502,14.820,18.237,15.276,14.183,15.225,15.210,15.887,19.317
GDP,191.000,198.000,207.000,217.000,224.000,233.000,249.000,256.000,262.000,267.000,...,339.000,344.000,351.000,358.000,368.000,380.000,389.000,397.000,399.000,428.000


Create new column, year2022, which contains values given a 0.05 growth rate prediction of every variable in year 2022:

In [23]:
all['value2022'] = all.apply(prediction, rate=1.05, axis=1)
all

,value1994,value1995,value1996,value1997,value1998,value1999,value2000,value2001,value2002,value2003,...,value2013,value2014,value2015,value2016,value2017,value2018,value2019,value2020,value2021,value2022
variables,,,,,,,,,,,,,,,,,,,,,
Total consumption,22.325,23.280,23.488,24.021,24.904,24.905,25.605,25.803,25.972,26.428,...,32.802,33.381,33.393,33.268,33.526,35.068,34.380,35.485,34.864,36.60720
"Electricity, gas and other fuels",11.551,11.786,13.261,14.056,14.669,14.579,15.426,17.058,17.774,17.939,...,24.282,24.322,22.306,20.812,23.919,23.940,23.580,19.925,25.635,26.91675
Purchase of vehicles,12.170,12.029,12.453,13.525,13.535,14.268,10.072,9.742,10.985,10.423,...,15.307,13.928,17.255,15.671,15.465,17.742,17.754,21.668,20.965,22.01325
Transport services,3.663,3.701,3.445,3.720,3.425,3.669,3.869,3.984,4.154,4.229,...,6.096,5.820,4.841,4.533,5.133,6.010,5.918,3.324,3.026,3.17730
Personal care,4.092,4.274,4.531,4.697,4.874,5.072,5.169,5.017,5.210,5.314,...,6.236,5.916,6.283,5.945,6.522,6.733,6.155,5.970,5.761,6.04905
Insurance,5.875,6.206,6.528,7.519,8.344,8.809,8.399,8.292,8.207,10.671,...,14.502,14.820,18.237,15.276,14.183,15.225,15.210,15.887,19.317,20.28285
GDP,191.000,198.000,207.000,217.000,224.000,233.000,249.000,256.000,262.000,267.000,...,344.000,351.000,358.000,368.000,380.000,389.000,397.000,399.000,428.000,449.40000


Check consumption of each variable over GDP:


In [24]:
for val in all.index:
    all.loc[val + "/GDP"] = all.loc[val] / all.loc["GDP"]
all

,value1994,value1995,value1996,value1997,value1998,value1999,value2000,value2001,value2002,value2003,...,value2013,value2014,value2015,value2016,value2017,value2018,value2019,value2020,value2021,value2022
variables,,,,,,,,,,,,,,,,,,,,,
Total consumption,22.325000,23.280000,23.488000,24.021000,24.904000,24.905000,25.605000,25.803000,25.972000,26.428000,...,32.802000,33.381000,33.393000,33.268000,33.526000,35.068000,34.380000,35.485000,34.864000,36.607200
"Electricity, gas and other fuels",11.551000,11.786000,13.261000,14.056000,14.669000,14.579000,15.426000,17.058000,17.774000,17.939000,...,24.282000,24.322000,22.306000,20.812000,23.919000,23.940000,23.580000,19.925000,25.635000,26.916750
Purchase of vehicles,12.170000,12.029000,12.453000,13.525000,13.535000,14.268000,10.072000,9.742000,10.985000,10.423000,...,15.307000,13.928000,17.255000,15.671000,15.465000,17.742000,17.754000,21.668000,20.965000,22.013250
Transport services,3.663000,3.701000,3.445000,3.720000,3.425000,3.669000,3.869000,3.984000,4.154000,4.229000,...,6.096000,5.820000,4.841000,4.533000,5.133000,6.010000,5.918000,3.324000,3.026000,3.177300
Personal care,4.092000,4.274000,4.531000,4.697000,4.874000,5.072000,5.169000,5.017000,5.210000,5.314000,...,6.236000,5.916000,6.283000,5.945000,6.522000,6.733000,6.155000,5.970000,5.761000,6.049050
Insurance,5.875000,6.206000,6.528000,7.519000,8.344000,8.809000,8.399000,8.292000,8.207000,10.671000,...,14.502000,14.820000,18.237000,15.276000,14.183000,15.225000,15.210000,15.887000,19.317000,20.282850
GDP,191.000000,198.000000,207.000000,217.000000,224.000000,233.000000,249.000000,256.000000,262.000000,267.000000,...,344.000000,351.000000,358.000000,368.000000,380.000000,389.000000,397.000000,399.000000,428.000000,449.400000
Total consumption/GDP,0.116885,0.117576,0.113469,0.110696,0.111179,0.106888,0.102831,0.100793,0.099130,0.098981,...,0.095355,0.095103,0.093277,0.090402,0.088226,0.090149,0.086599,0.088935,0.081458,0.081458
"Electricity, gas and other fuels/GDP",0.060476,0.059525,0.064063,0.064774,0.065487,0.062571,0.061952,0.066633,0.067840,0.067187,...,0.070587,0.069293,0.062307,0.056554,0.062945,0.061542,0.059395,0.049937,0.059895,0.059895


Set decimal units:

In [25]:
all = all.astype(float).round(decimals=2)
all

,value1994,value1995,value1996,value1997,value1998,value1999,value2000,value2001,value2002,value2003,...,value2013,value2014,value2015,value2016,value2017,value2018,value2019,value2020,value2021,value2022
variables,,,,,,,,,,,,,,,,,,,,,
Total consumption,22.32,23.28,23.49,24.02,24.90,24.90,25.60,25.80,25.97,26.43,...,32.80,33.38,33.39,33.27,33.53,35.07,34.38,35.48,34.86,36.61
"Electricity, gas and other fuels",11.55,11.79,13.26,14.06,14.67,14.58,15.43,17.06,17.77,17.94,...,24.28,24.32,22.31,20.81,23.92,23.94,23.58,19.92,25.64,26.92
Purchase of vehicles,12.17,12.03,12.45,13.52,13.54,14.27,10.07,9.74,10.98,10.42,...,15.31,13.93,17.26,15.67,15.46,17.74,17.75,21.67,20.96,22.01
Transport services,3.66,3.70,3.44,3.72,3.42,3.67,3.87,3.98,4.15,4.23,...,6.10,5.82,4.84,4.53,5.13,6.01,5.92,3.32,3.03,3.18
Personal care,4.09,4.27,4.53,4.70,4.87,5.07,5.17,5.02,5.21,5.31,...,6.24,5.92,6.28,5.94,6.52,6.73,6.16,5.97,5.76,6.05
Insurance,5.88,6.21,6.53,7.52,8.34,8.81,8.40,8.29,8.21,10.67,...,14.50,14.82,18.24,15.28,14.18,15.22,15.21,15.89,19.32,20.28
GDP,191.00,198.00,207.00,217.00,224.00,233.00,249.00,256.00,262.00,267.00,...,344.00,351.00,358.00,368.00,380.00,389.00,397.00,399.00,428.00,449.40
Total consumption/GDP,0.12,0.12,0.11,0.11,0.11,0.11,0.10,0.10,0.10,0.10,...,0.10,0.10,0.09,0.09,0.09,0.09,0.09,0.09,0.08,0.08
"Electricity, gas and other fuels/GDP",0.06,0.06,0.06,0.06,0.07,0.06,0.06,0.07,0.07,0.07,...,0.07,0.07,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06


# Graphs:

Start by resetting the index:

In [26]:
all = all.reset_index()
all

,variables,value1994,value1995,value1996,value1997,value1998,value1999,value2000,value2001,value2002,...,value2013,value2014,value2015,value2016,value2017,value2018,value2019,value2020,value2021,value2022
0,Total consumption,22.32,23.28,23.49,24.02,24.90,24.90,25.60,25.80,25.97,...,32.80,33.38,33.39,33.27,33.53,35.07,34.38,35.48,34.86,36.61
1,"Electricity, gas and other fuels",11.55,11.79,13.26,14.06,14.67,14.58,15.43,17.06,17.77,...,24.28,24.32,22.31,20.81,23.92,23.94,23.58,19.92,25.64,26.92
2,Purchase of vehicles,12.17,12.03,12.45,13.52,13.54,14.27,10.07,9.74,10.98,...,15.31,13.93,17.26,15.67,15.46,17.74,17.75,21.67,20.96,22.01
3,Transport services,3.66,3.70,3.44,3.72,3.42,3.67,3.87,3.98,4.15,...,6.10,5.82,4.84,4.53,5.13,6.01,5.92,3.32,3.03,3.18
4,Personal care,4.09,4.27,4.53,4.70,4.87,5.07,5.17,5.02,5.21,...,6.24,5.92,6.28,5.94,6.52,6.73,6.16,5.97,5.76,6.05
5,Insurance,5.88,6.21,6.53,7.52,8.34,8.81,8.40,8.29,8.21,...,14.50,14.82,18.24,15.28,14.18,15.22,15.21,15.89,19.32,20.28
6,GDP,191.00,198.00,207.00,217.00,224.00,233.00,249.00,256.00,262.00,...,344.00,351.00,358.00,368.00,380.00,389.00,397.00,399.00,428.00,449.40
7,Total consumption/GDP,0.12,0.12,0.11,0.11,0.11,0.11,0.10,0.10,0.10,...,0.10,0.10,0.09,0.09,0.09,0.09,0.09,0.09,0.08,0.08
8,"Electricity, gas and other fuels/GDP",0.06,0.06,0.06,0.06,0.07,0.06,0.06,0.07,0.07,...,0.07,0.07,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06
9,Purchase of vehicles/GDP,0.06,0.06,0.06,0.06,0.06,0.06,0.04,0.04,0.04,...,0.04,0.04,0.05,0.04,0.04,0.05,0.04,0.05,0.05,0.05


Transform dataframe from wide to long format:

In [27]:
all_long = pd.wide_to_long(all, stubnames='value', i='variables', j='year')
all_long

,,value
variables,year,
Total consumption,1994,22.32
"Electricity, gas and other fuels",1994,11.55
Purchase of vehicles,1994,12.17
Transport services,1994,3.66
Personal care,1994,4.09
...,...,...
Purchase of vehicles/GDP,2022,0.05
Transport services/GDP,2022,0.01
Personal care/GDP,2022,0.01


Save a copy of the final format of our dataset (uncomment to run the code):

In [28]:
# all_long.to_csv('data/FU07_cp_long.xlsx', index=False)

Reset the index again:

In [29]:
all_long = all_long.reset_index()
all_long

,variables,year,value
0,Total consumption,1994,22.32
1,"Electricity, gas and other fuels",1994,11.55
2,Purchase of vehicles,1994,12.17
3,Transport services,1994,3.66
4,Personal care,1994,4.09
...,...,...,...
401,Purchase of vehicles/GDP,2022,0.05
402,Transport services/GDP,2022,0.01
403,Personal care/GDP,2022,0.01
404,Insurance/GDP,2022,0.05


Plotting a interactive graph:

In [30]:
# Static dataframes
# all_consumption = all_long.loc[all_long['variables'] == 'Total consumption', :]
# all_gdp = all_long.loc[all_long['variables'] == 'GDP', :]

# # Plot the content of the data frame
# all_consumption.plot(x='year',y='value',legend=False);
# all_gdp.plot(x='year',y='value',legend=False);

import ipywidgets as widgets
def plot_e(df, variable): 
    I = df['variables'] == variable
    ax=df.loc[I,:].plot(x='year', y='value', style='-o', legend=False)

widgets.interact(plot_e, 
    df = widgets.fixed(all_long),
    variable = widgets.Dropdown(description='variables', 
                                    options=all_long.variables.unique(), 
                                    value='Total consumption')
); 


interactive(children=(Dropdown(description='variables', options=('Total consumption', ' Electricity, gas and o…

Next things to do: 

- **Solving problem about real prices base year:** <br> 
    Base year for gdp = 2010 <br>
    Base year for cop = 2015 <br>
   We solve the problem by using current prices. - <font color=green> Done </font> <br>
   **Problem with current prices:** GDP is in million of DKK while consumption is just in DKK - <font color=green> Done </font> <br>
   Solved with:
   - changed parameters: gdp is now in current per capita prices (so no million DKK problem);<br>
   - imported new excel file FU07_cp: cop is now at current prices
- Drop remaining NaN rows in cop dataset - <font color=green>Done</font>
- New index for cop dataset - <font color=green>Done</font>
- Trasform gdp dataset: 
    - years as column names - <font color=green>Done</font>
    - only one row called gdp: drop other - <font color=green>Done</font>
- Concat. datasets - <font color=green>Done</font> 
- Create new data as a result of running an operator on other data in the dataset:
    (e.g. summing two rows) - <font color=green>Done</font> (But the comment should be considered)
- Running a function on the dataset - <font color=red>To do</font>
- Modify GDP values of Consumption value so that is there homogeneity of unit measure (now: Consumption is in dkk and and GDP in 1000 Dkk) - <font color=green>Done</font> (albeit the other way around)
- New row: difference between consumption and gdp = ??? look documentation
- Graphs - <font color=green>Done</font> 
- Adding information about unit measure of the dataset (1000 DKK)
- Bonus: Measure of consumption/GDP is too low; can it be because we are using current prices?